In [1]:
#this imports all the commands needed for the script to work#
import os
import numpy as np
import nilearn
import glob
#import matplotlib
import nibabel as nib
from nilearn.image import concat_imgs, index_img, smooth_img
from nilearn.image import resample_to_img
#from nilearn import plotting
from nilearn.input_data import NiftiMasker
from sklearn.svm import SVC
from sklearn.cross_validation import LeaveOneLabelOut, cross_val_score, permutation_test_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from nilearn import image
#from nilearn.plotting import plot_stat_map, show
from sklearn.dummy import DummyClassifier


/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/nikkibytes/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functio

In [2]:
# ---STEP 1---
#Cncatenate the imagine data into a NIFTI-2 file. 
#Note: the data does not fit into a NIFTI-1 file, due to large number of subs. 

#set basepath
basepath=os.path.join('/projects','niblab','data','eric_data','W1','imagine')

#make a list of the files to concat
all_func = glob.glob(os.path.join(basepath,'level1_grace_edit','cs*++.feat','filtered_func_data.nii.gz'))
half_func = all_func[:67]
#load in all the files from the glob above, then convert them from nifti1 to nifti2
ni2_funcs = (nib.Nifti2Image.from_image(nib.load(func)) for func in half_func)
#concat, this is with nibabel, but should work with nilearn too
ni2_concat = nib.concat_images(ni2_funcs, check_affines=False, axis=3)
#set the output file name
outfile=os.path.join(basepath,'concatenated_imagine.nii')
#write the file
ni2_concat.to_filename(outfile)

# ---STEP 2---
#load & prepare MRI data
#load, fxnl, anatomical, & mask for plotting
fmri_subjs=os.path.join(basepath, 'concatenated_imagine.nii')
average_ana=os.path.join(basepath,'CS_avg_mprage_image.nii.gz')
imag_mask=os.path.join('/projects','niblab','scripts','nilean_stuff','power_roimask_4bi.nii.gz')

#plot mask (Power ROIs) over anatomical that is defined above
#plotting.plot_roi(imag_mask,bg_img=average_ana,cmap='Paired')

#load labels for the functional data
stim = os.path.join('/projects','niblab','scripts','nilean_stuff','label_all_sub.csv')
labels = np.recfromcsv(stim, delimiter=",")
print(labels)
#Its shape corresponds to the number of time-points times the number of voxels in the mask

#Retrieve the behavioral targets, that we are going to predict in the decoding
y_mask = labels['label']
subs = labels['sub']



sizeof_hdr should be 540; set sizeof_hdr to 540
sizeof_hdr should be 540; set sizeof_hdr to 540
